In [ ]:
### 1. Setup and Data Loading

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Load the Titanic dataset
df = pd.read_csv('Module.csv') 
# Ensure 'titanic.csv' is in the same directory as your Jupyter Notebook or provide the full path.

print("Original DataFrame Head:")
print(df.head())
print("\nOriginal DataFrame Info:")
df.info()

In [ ]:
### 2. Train/Test Split

X = df.drop('survived', axis=1)
y = df['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

In [ ]:
### 3. Preprocessing for Columns (Handling Missing Values, Scaling, Encoding)

#### Separate Column Types

numeric_features = ['age', 'fare', 'pclass']
categorical_features = ['sex', 'embarked']

In [ ]:
#### Numeric Pipeline: Handle Missing Values + Scale

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),#Takes thee mean of the row of that NaN point and replace its value by the mean value.
    ('scaler', StandardScaler())#StandardScaler transforms each feature so that it has a mean of 0 and a standard deviation of 1.
])

#### Categorical Pipeline: Handle Missing Values + Encode

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),# Fillups or replaces the NaN or null value with the most frequent value.
    ('encoder', OneHotEncoder(handle_unknown='ignore'))#This encoder converts the categories into a numerical format. It creates new binary (0 or 1) columns for each unique category.
])

In [ ]:
#### Combine into Column Transformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
### 4. Model Definition and Training (Fit/Predict)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
### 5. Evaluation
#### Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f}")

In [ ]:
#### Confusion Matrix

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

In [ ]:
#### Classification Report

print("\nClassification Report:")
print(classification_report(y_test, y_pred))